In [77]:
import pandas as pd
import plotly.express as px
import numpy as np


def bootstrap_confidence_interval(values, *, n_boot: int = 1000, ci: int = 95, seed: int = 42):
    """
    Compute the bootstrap confidence interval of the mean of a numeric variable.

    Args:
        values (1-D array-like of numeric values)
        n_boot (int): number of bootstrap resamples
        ci (int): confidence level, e.g. 95 for 95 % CI
        seed (int): seed for reproducibility
    """
    assert 0 < ci < 100, "CI must be between 0 and 100"
    values = np.asarray(values, dtype=float)
    n = values.size
    # Handle edge case where there are no samples
    if n == 0:
        return np.nan, (np.nan, np.nan)

    rng = np.random.default_rng(seed)

    # Draw `n_boot` resamples of size `n`
    resample_idxs = rng.choice(n, size=(n_boot, n), replace=True)
    resample_means = values[resample_idxs].mean(axis=1)

    # Percentile-based CI
    alpha = (100 - ci) / 2  # e.g. 2.5 for a 95 % CI
    lower, upper = np.percentile(resample_means, [alpha, 100 - alpha])
    mean = values.mean()

    return mean, (lower, upper)


def analyze_results(file_name):
    full_results = pd.read_json(file_name)

    aggregated_results = {}
    for column in full_results.columns:
        mean, (lower, upper) = bootstrap_confidence_interval(full_results[column])
        aggregated_results[column] = [mean, mean - lower, upper - mean]

    aggregated_results = (
        pd.DataFrame.from_dict(aggregated_results, orient="index", columns=["mean", "lower", "upper"])
        .reset_index()
        .rename(columns={"index": "n_conversions"})
    )
    # aggregated_results = aggregated_results.melt(value_vars=["lower", "upper", "mean"])
    return aggregated_results


results_json = analyze_results("results/benchmark_results_json.json")
results_json["agent_type"] = "json"

results_code = analyze_results("results/benchmark_results_code.json")
results_code["agent_type"] = "code"

results = pd.concat([results_code, results_json], axis=0)

display(results)

,n_conversions,mean,lower,upper,agent_type
0,10,0.87,0.06,0.06,code
1,20,0.86,0.08,0.06,code
2,30,0.84,0.08,0.06,code
3,40,0.83,0.08,0.07,code
4,50,0.81,0.09,0.07,code
5,60,0.79,0.08,0.07,code
6,70,0.86,0.07,0.07,code
7,80,0.82,0.08,0.07,code
8,90,0.82,0.08,0.07,code
9,100,0.76,0.09,0.08,code


In [91]:
error_y = ("upper_error",)
error_y_minus = ("lower_error",)

fig = px.bar(
    results,
    x="n_conversions",
    y="mean",
    error_y="upper",
    error_y_minus="lower",
    color_continuous_scale="viridis",
    color="agent_type",
)


fig.update_layout(
    title="<b>Comparing the reliability of code and JSON agents on repeated operations</b>",
    xaxis_title="<b>Number of Operations</b>",
    yaxis_title="<b>Success Rate</b>",
    barmode="group",
    width=1000,
    height=500,
    bargap=0.4,
)
fig.show()
